# MySQL Ranking Functions Guide

## Overview

MySQL provides three window functions for ranking rows: `RANK()`, `DENSE_RANK()`, and `ROW_NUMBER()`. All require MySQL 8.0+.

---


## RANK()

### Behavior
- Assigns the same rank to tied values
- **Skips ranks** after ties (leaves gaps)
- Format: `RANK() OVER (ORDER BY column [DESC])`

### Example
```sql
    SELECT 
        name,
        score,
        RANK() OVER (ORDER BY score DESC) AS rank
    FROM students;
```

**Output:**
```text
    name    | score | rank
    --------|-------|-----
    Alice   | 95    | 1
    Bob     | 95    | 1    ← tied
    Carol   | 90    | 3    ← skips 2
    David   | 85    | 4
    Eve     | 85    | 4    ← tied
    Frank   | 80    | 6    ← skips 5
```

### Use Cases
- Olympic-style rankings (gold, gold, bronze)
- Competition leaderboards where ties matter
- When gaps in ranking are acceptable

---



## DENSE_RANK()

### Behavior
- Assigns the same rank to tied values
- **No gaps** in ranking sequence
- Format: `DENSE_RANK() OVER (ORDER BY column [DESC])`

### Example
```sql
    SELECT 
        name,
        score,
        DENSE_RANK() OVER (ORDER BY score DESC) AS dense_rank
    FROM students;
```

**Output:**
```text
    name    | score | dense_rank
    --------|-------|------------
    Alice   | 95    | 1
    Bob     | 95    | 1          ← tied
    Carol   | 90    | 2          ← no skip
    David   | 85    | 3
    Eve     | 85    | 3          ← tied
    Frank   | 80    | 4          ← no skip
```

### Use Cases
- Grade level rankings (A, A, B, C - no gaps)
- Top-N within categories
- When continuous ranking sequence is important

---



## ROW_NUMBER()

### Behavior
- Assigns **unique** sequential numbers
- No ties - breaks them arbitrarily
- Format: `ROW_NUMBER() OVER (ORDER BY column [DESC])`

### Example
```sql
SELECT 
    name,
    score,
    ROW_NUMBER() OVER (ORDER BY score DESC) AS row_num
FROM students;
```

**Output:**
```
name    | score | row_num
--------|-------|--------
Alice   | 95    | 1
Bob     | 95    | 2       ← no tie, arbitrary order
Carol   | 90    | 3
David   | 85    | 4
Eve     | 85    | 5       ← no tie, arbitrary order
Frank   | 80    | 6
```

### Use Cases
- Pagination (getting rows 11-20)
- Selecting exactly one row per group
- When you need unique identifiers regardless of ties

---

## Comparison Table

| Function | Handles Ties? | Gaps in Sequence? | Output for [95, 95, 90, 85] |
|----------|---------------|-------------------|------------------------------|
| `RANK()` | Yes (same rank) | Yes | 1, 1, 3, 4 |
| `DENSE_RANK()` | Yes (same rank) | No | 1, 1, 2, 3 |
| `ROW_NUMBER()` | No (unique) | No | 1, 2, 3, 4 |

---



## Partitioning (Ranking Within Groups)

All three functions support `PARTITION BY` to rank within subgroups:

```sql
SELECT 
    department,
    name,
    salary,
    RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS dept_rank,
    DENSE_RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS dept_dense_rank
FROM employees;
```

**Output:**
```
department | name  | salary | dept_rank | dept_dense_rank
-----------|-------|--------|-----------|----------------
Sales      | Alice | 80000  | 1         | 1
Sales      | Bob   | 80000  | 1         | 1
Sales      | Carol | 75000  | 3         | 2
IT         | David | 90000  | 1         | 1
IT         | Eve   | 85000  | 2         | 2
```

---



## Common Pitfalls

### 1. Using Reserved Keywords as Aliases
```sql
-- ❌ ERROR: 'rank' is a reserved keyword
SELECT score, RANK() OVER (ORDER BY score) AS rank FROM Scores;

-- ✅ Solution 1: Use backticks
SELECT score, RANK() OVER (ORDER BY score) AS `rank` FROM Scores;

-- ✅ Solution 2: Use different alias
SELECT score, RANK() OVER (ORDER BY score) AS score_rank FROM Scores;
```

### 2. Missing OVER Clause
```sql
-- ❌ ERROR
SELECT RANK() FROM students;

-- ✅ CORRECT
SELECT RANK() OVER (ORDER BY score DESC) FROM students;
```

### 3. Version Compatibility
Window functions require **MySQL 8.0 or higher**. For older versions, use subqueries or session variables.

---



## Advanced Examples

### Top 3 Per Category
```sql
WITH RankedProducts AS (
    SELECT 
        category,
        product_name,
        sales,
        DENSE_RANK() OVER (PARTITION BY category ORDER BY sales DESC) AS rank
    FROM products
)
SELECT * FROM RankedProducts WHERE rank <= 3;
```

### Filtering by Rank
```sql
SELECT * FROM (
    SELECT 
        name,
        score,
        RANK() OVER (ORDER BY score DESC) AS rank
    FROM students
) AS ranked
WHERE rank <= 10;  -- Top 10 students
```

### Multiple Ordering Criteria
```sql
SELECT 
    name,
    score,
    age,
    RANK() OVER (ORDER BY score DESC, age ASC) AS rank
FROM students;
-- Ties in score are broken by age (younger gets higher rank)
```

---



## Quick Reference

**Choose `RANK()` when:**
- You want Olympic-style rankings (1st, 1st, 3rd)
- Gaps after ties are acceptable

**Choose `DENSE_RANK()` when:**
- You need continuous rankings (1st, 1st, 2nd)
- No gaps in sequence required

**Choose `ROW_NUMBER()` when:**
- You need unique row identifiers
- Ties should be broken arbitrarily
- Implementing pagination